In [1]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
from torch.utils.data import DataLoader

from dataset.dataset_loader import dataset_loader
import keras_tuner

from utils.my_tuner import HistoryDeletingBayesianOptimization

from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper

In [3]:
import torch

torch.cuda.is_available()

True

In [5]:
# Initial steps
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((192, 192), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=32, shuffle=True)

# Steps:

- 1: Study the best network inner structure by searching for best hidden nodes structure.
- 2: Study the best Hyperparameters for SGD of our top 4 models in general and also the best overfitting one
- 3: See top results and try to increase performance by attaching an Augmentation procedure.
- 4: See if adding dropout increases the performance of model

# Step 1:  Study the best network inner structure by searching for best hidden nodes structure

  As we need to find the structure of the two layers network. We already saw that 3 layers are harder to train and didn't yield a great performance result.
  
  We still will try the top 3 models of the 3 layer study with augmentation and dropout to see if performance increases.
```py  
TwoHiddenLayersTunableAugmentedDNN() # Family we will be tuning
```

In [6]:
from models.structure.tunable_model_family_hypermodel import TunableModelFamilyHypermodel
from models.naive_dnn_gen.two_layers_dnn import TwoHiddenLayersTunableAugmentedDNN

# For now the optimizer is also fixed to SGD with these parameters:
hyperparameters.Fixed("lr", 1e-4)
hyperparameters.Fixed("momentum", 0.9)

# For now dropout layers are frozen to be disabled.
hyperparameters.Fixed("dropout_0", False)
hyperparameters.Fixed("dropout_1", False)

project_name = "two-layers-192-192"
project_directory = "dnn-search"

tuner = HistoryDeletingBayesianOptimization(
    TunableModelFamilyHypermodel((3, 192, 192), TwoHiddenLayersTunableAugmentedDNN()),
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=True,
    executions_per_trial=2,
    overwrite=False,
    directory=project_directory,
    max_trials=15,
    project_name=project_name
)

In [7]:
tuner.search(train_dataloader, epochs=20, validation_data=validation_dataloader, callbacks=[keras.callbacks.CSVLogger(
    f"{project_directory}/{project_name}/search.log", separator=",", append=True)
])

Trial 6 Complete [00h 16m 36s]

Best val_loss So Far: 0.441125825047493
Total elapsed time: 02h 03m 55s

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
0.0001            |0.0001            |lr
0.9               |0.9               |momentum
False             |False             |dropout_0
False             |False             |dropout_1
2560              |1536              |units_0
128               |384               |units_1

Epoch 1/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 48s 403ms/step - accuracy: 0.5943 - loss: 0.6634 - val_accuracy: 0.6748 - val_loss: 0.6064
Epoch 2/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 49s 411ms/step - accuracy: 0.7162 - loss: 0.5685 - val_accuracy: 0.6990 - val_loss: 0.5602
Epoch 3/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 44s 372ms/step - accuracy: 0.7525 - loss: 0.5110 - val_accuracy: 0.6959 - val_loss: 0.5950
Epoch 4/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 48s 407ms/step - accuracy: 0.7556 - loss: 0.5032 - val_accuracy: 0.7656 - val_loss: 0.4907
Epoch 5/20
119/

Traceback (most recent call last):
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jacopo/PycharmProjects/muffin-stat-project/utils/my_tuner.py", line 20, in run_trial
    return super().run_trial(trial, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jacopo/PycharmProjects/muffin-stat-project/utils/my_tuner.py", line 20, in run_trial
    return super().run_trial(trial, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, 

In [9]:
from models.structure.base_model_family import HiddenLayerStructure
from models.naive_dnn_gen.two_layers_dnn import TwoHiddenLayersDNNAugModelFamily

family_gen = TwoHiddenLayersDNNAugModelFamily()
family_gen.hidden_layer_0 = HiddenLayerStructure(2718, None)
family_gen.hidden_layer_1 = HiddenLayerStructure(728, None)

model = family_gen.make_model((3, 192, 192))

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9, nesterov=True), metrics=['accuracy'])

model.summary(expand_nested=True)

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ AugmentedDNN (InputLayer)       │ (None, 3, 192, 192)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_13 (Functional)      │ (None, 192, 192, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ AugmentedDNN (InputLayer)  │ (None, 3, 192, 192)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ permute_2 (Permute)        │ (None, 192, 192, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ random_contrast_2          │ (None, 192, 192, 3)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ random_flip_2 (RandomFlip) │ (None, 192, 192, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_15 (Functional)      │ (None, 1)              │   302,571,935 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ AugmentedDNN (InputLayer)  │ (None, 192, 192, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ flatten_2 (Flatten)        │ (None, 110592)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_6 (Dense)            │ (None, 2718)           │   300,591,774 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_7 (Dense)            │ (None, 728)            │     1,979,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_8 (Dense)            │ (None, 1)              │           729 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 302,571,935 (1.13 GB)

 Trainable params: 302,571,935 (1.13 GB)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(train_dataloader, validation_data=validation_dataloader, epochs=150,
                    callbacks=[keras.callbacks.EarlyStopping(
                        monitor='val_loss', min_delta=1e-4, patience=20, verbose=1, mode='min',
                        restore_best_weights=True
                    )])

Epoch 1/150
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 394ms/step - accuracy: 0.5857 - loss: 0.6649 - val_accuracy: 0.7159 - val_loss: 0.5835
Epoch 2/150
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 384ms/step - accuracy: 0.7110 - loss: 0.5601 - val_accuracy: 0.6874 - val_loss: 0.5902
Epoch 3/150
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 385ms/step - accuracy: 0.7229 - loss: 0.5400 - val_accuracy: 0.6737 - val_loss: 0.6034
Epoch 4/150
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 389ms/step - accuracy: 0.7474 - loss: 0.5288 - val_accuracy: 0.7497 - val_loss: 0.5147
Epoch 5/150
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 391ms/step - accuracy: 0.7579 - loss: 0.5108 - val_accuracy: 0.7878 - val_loss: 0.4904
Epoch 6/150
119/119 ━━━━━━━━━━━━━━━━━━━━ 47s 395ms/step - accuracy: 0.7683 - loss: 0.4891 - val_accuracy: 0.8025 - val_loss: 0.4724
Epoch 7/150
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 387ms/step - accuracy: 0.7711 - loss: 0.4825 - val_accuracy: 0.7677 - val_loss: 0.4916
Epoch 8/150
119/119 ━━━━━━━━━━━━━━━━━━━━ 46s 384ms/step - accuracy: 0.7668 -